In [13]:
from bs4 import BeautifulSoup as bs
from tqdm import tqdm
import pandas as pd
import requests
import json
import re

In [14]:
headers = {
    'Accept': 'application/json, text/javascript, */*; q=0.01',
    'Accept-Language': 'ko,en;q=0.9,en-US;q=0.8',
    'Content-Type': 'application/json;charset=UTF-8',
    'Host': 'www.bigkinds.or.kr',
    'Origin': 'https://www.bigkinds.or.kr',
    'Referer': 'https://www.bigkinds.or.kr/v2/news/index.do',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36 Edg/111.0.1661.62',
}

In [20]:
# payload, .기사 리스트 크롤링
def request_data(keyword, start_date, end_date):
  data = {
    "indexName":"news",
    "searchKey":f"{keyword}",
    "searchKeys":[{}],
    "byLine":"",
    "searchFilterType":"1",
    "searchScopeType":"1",
    "searchSortType":"date",
    "sortMethod":"date",
    "mainTodayPersonYn":"",
    "startDate":f"{start_date}",
    "endDate":f"{end_date}",
    "newsIds":[],
    "categoryCodes":[],
    "providerCodes":[],
    "incidentCodes":[],
    "networkNodeType":"",
    "topicOrigin":"",
    "dateCodes":[],
    "editorialIs":"false",
    "startNo":1,
    "resultNumber":1000000,
    "isTmUsable":"false",
    "isNotTmUsable":"false"
  }
  return json.dumps(data).encode('utf-8')

In [21]:
# 각 기사별 내용 크롤링
def request_content(new_id):
  url = 'https://www.bigkinds.or.kr/news/detailView.do'
  params = {
    'docId': f'{new_id}',
    'returnCnt': '1',
    'sectionDiv': '1000',
  }
  
  result = requests.get(url, headers=headers, params=params).json()
  return result['detail']['CONTENT']

In [22]:
# all function
def news_crawl(keyword, start_date, end_date):
  url = 'https://www.bigkinds.or.kr/api/news/search.do'
  headers = {
    'Accept': 'application/json, text/javascript, */*; q=0.01',
    'Accept-Language': 'ko,en;q=0.9,en-US;q=0.8',
    'Content-Type': 'application/json;charset=UTF-8',
    'Host': 'www.bigkinds.or.kr',
    'Origin': 'https://www.bigkinds.or.kr',
    'Referer': 'https://www.bigkinds.or.kr/v2/news/index.do',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36 Edg/111.0.1661.62',
}
  data = request_data(keyword, start_date, end_date)
  res = requests.post(url, headers=headers, data=data).json()
  data_lst = []
  for i in tqdm(res['resultList']):
    title = i['TITLE']
    where = i['PROVIDER']
    date=i['DATE']
    year = date[0:4]
    month = date[4:6]
    url = i['PROVIDER_LINK_PAGE']
    content = request_content(i['NEWS_ID']).replace('<br/>', ' ')
    content = re.sub('(. [가-힣]*=.*)', '', content).strip()
    data_lst.append([title,where,year,month, url, content])
    df = pd.DataFrame(data_lst, columns=['기사제목', '언론사','년도','월','url', '기사내용'])
  return df

In [23]:
news_crawl("태양","2020-01-01","2022-04-01")

  1%|          | 139/19322 [00:17<40:50,  7.83it/s]  


KeyboardInterrupt: 